# 🎯 XGBoost Hyperparameter Tuning

**Team:** Error 400  
**Dataset:** UK Housing Prices (1995-2017)  
**Purpose:** Fine-tune XGBoost model to achieve best possible performance

---

## 📋 What is Hyperparameter Tuning?

Our XGBoost model has many "settings" (hyperparameters) that control how it learns:
- **n_estimators** - How many trees to build
- **max_depth** - How deep each tree can grow
- **learning_rate** - How fast the model learns
- And many more...

**Default settings** gave us R² = 63%. But with the **right settings**, we can do even better!

---

## 🎯 Our Goal

- **Current Performance:** R² = 63.0%, MAE = £50,689
- **Target:** R² > 65%, MAE < £48,000

Let's find the best hyperparameters! 🚀

---

## 📚 Step 1: Setup Environment

Import required libraries and load our clean data.

In [ ]:
# ==========================================
# Import Libraries
# ==========================================

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# XGBoost
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder  # ← MAKE SURE THIS IS HERE!

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyCaret
from pycaret.regression import *

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# ==========================================
# Load and Prepare Data
# ==========================================

print("📂 Loading clean data...")
df = pd.read_parquet("data/cleaned/housing_FULL_clean.parquet")

print(f"✅ Loaded {len(df):,} records")
print(f"📅 Date range: {df['year'].min()} - {df['year'].max()}")
print(f"💰 Price range: £{df['price'].min():,} - £{df['price'].max():,}")

df.head()

📂 Loading clean data...
✅ Loaded 5,972,822 records
📅 Date range: 1995 - 2017
💰 Price range: £6,000 - £1,320,509


,price,town_city,district,county,year,month,quarter,day_of_week,property_type_label,is_new_build,tenure_label
0,18899,WAKEFIELD,LEEDS,WEST YORKSHIRE,1995,6,2,4,Semi-Detached,False,Freehold
1,82000,GREENFORD,EALING,GREATER LONDON,1995,6,2,4,Semi-Detached,False,Freehold
2,55000,SWINDON,THAMESDOWN,THAMESDOWN,1995,3,1,4,Semi-Detached,False,Freehold
3,28500,BOOTLE,SEFTON,MERSEYSIDE,1995,10,4,3,Terraced,False,Freehold
4,76000,LEEDS,LEEDS,WEST YORKSHIRE,1995,1,1,2,Detached,False,Freehold


---

## 🔧 Step 2: Prepare Features

Apply the same feature engineering we used in PyCaret:
1. Create smart location encoding (top towns/districts)
2. Select relevant features
3. Encode categorical variables

In [6]:
# ==========================================
# Feature Engineering
# ==========================================

print("🔧 Creating engineered features...\n")

# 1. Smart location encoding
top_towns = df['town_city'].value_counts().head(30).index
df['town_encoded'] = df['town_city'].apply(lambda x: x if x in top_towns else 'OTHER')

top_districts = df['district'].value_counts().head(50).index
df['district_encoded'] = df['district'].apply(lambda x: x if x in top_districts else 'OTHER')

print(f"✓ Towns: {df['town_city'].nunique()} → {df['town_encoded'].nunique()} categories")
print(f"✓ Districts: {df['district'].nunique()} → {df['district_encoded'].nunique()} categories")

# 2. Select features
feature_columns = [
    'property_type_label',
    'is_new_build',
    'tenure_label',
    'county',
    'district_encoded',
    'town_encoded',
    'year',
    'month',
    'quarter'
]

X = df[feature_columns].copy()
y = df['price'].copy()

print(f"\n✅ Features prepared: {X.shape}")
print(f"🎯 Target: {y.shape}")

🔧 Creating engineered features...

✓ Towns: 1167 → 31 categories
✓ Districts: 453 → 51 categories

✅ Features prepared: (5972822, 9)
🎯 Target: (5972822,)


---

## 🔢 Step 3: Encode Categorical Variables & Split Data

Convert text categories to numbers and split into train/test sets.

In [ ]:
# ==========================================
# Encode Categorical Variables
# ==========================================

print("🔢 Encoding categorical variables...\n")

X_encoded = X.copy()

# Convert boolean to int
X_encoded['is_new_build'] = X_encoded['is_new_build'].astype(int)

# Encode categorical columns
categorical_cols = ['property_type_label', 'tenure_label', 'county', 'district_encoded', 'town_encoded']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"  ✓ Encoded {col}: {len(le.classes_)} categories")

print(f"\n✅ Encoding complete! Shape: {X_encoded.shape}")

# ==========================================
# Train/Test Split
# ==========================================

print("\n✂️ Splitting data...")

# To this:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,  # ✅ Use original X with strings
    test_size=0.2,
    random_state=42
)

print(f"✅ Training set: {len(X_train):,} records")
print(f"✅ Test set: {len(X_test):,} records")
print(f"\n📊 Ready for tuning!")

🔢 Encoding categorical variables...

  ✓ Encoded property_type_label: 5 categories
  ✓ Encoded tenure_label: 3 categories
  ✓ Encoded county: 127 categories
  ✓ Encoded district_encoded: 51 categories
  ✓ Encoded town_encoded: 31 categories

✅ Encoding complete! Shape: (5972822, 9)

✂️ Splitting data...
✅ Training set: 4,778,257 records
✅ Test set: 1,194,565 records

📊 Ready for tuning!


In [8]:
# ==========================================
# Baseline XGBoost Model
# ==========================================

print("🌲 Training baseline XGBoost with default parameters...\n")

# Train with defaults
baseline_model = XGBRegressor(
    random_state=42,
    n_jobs=-1
)

baseline_model.fit(X_train, y_train)

# Predictions
y_pred_baseline = baseline_model.predict(X_test)

# Calculate metrics
baseline_r2 = r2_score(y_test, y_pred_baseline)
baseline_mae = mean_absolute_error(y_test, y_pred_baseline)
baseline_rmse = np.sqrt(mean_squared_error(y_test, y_pred_baseline))

print("="*60)
print("📊 BASELINE XGBOOST PERFORMANCE (Default Parameters)")
print("="*60)
print(f"R² Score:                    {baseline_r2:.4f} ({baseline_r2*100:.2f}%)")
print(f"Mean Absolute Error (MAE):   £{baseline_mae:,.0f}")
print(f"Root Mean Squared Error:     £{baseline_rmse:,.0f}")
print("="*60)

print("\n💡 This is our starting point. Let's improve it with tuning!")

🌲 Training baseline XGBoost with default parameters...

📊 BASELINE XGBOOST PERFORMANCE (Default Parameters)
R² Score:                    0.5860 (58.60%)
Mean Absolute Error (MAE):   £52,424
Root Mean Squared Error:     £91,221

💡 This is our starting point. Let's improve it with tuning!


In [9]:
print(f"Records loaded: {len(df):,}")
print(f"Price stats:")
print(df['price'].describe())
print(f"\nPrice range: £{df['price'].min():,} - £{df['price'].max():,}")

Records loaded: 5,972,822
Price stats:
count    5.972822e+06
mean     1.666644e+05
std      1.424276e+05
min      6.000000e+03
25%      7.500000e+04
50%      1.300000e+05
75%      2.100000e+05
max      1.320509e+06
Name: price, dtype: float64

Price range: £6,000 - £1,320,509


---

## 🎯 Step 5: Hyperparameter Tuning

We'll use **RandomizedSearchCV** to test different combinations of hyperparameters:

**Key Parameters to Tune:**
- **n_estimators** - Number of trees (more = better but slower)
- **max_depth** - Tree depth (deeper = more complex)
- **learning_rate** - How fast model learns (lower = slower but better)
- **subsample** - Fraction of data used per tree
- **colsample_bytree** - Fraction of features used per tree
- **min_child_weight** - Minimum data in leaf nodes

**Method:** Test 50 random combinations, keep the best!

In [10]:
# ==========================================
# Hyperparameter Tuning - Smart Approach for Large Dataset
# ==========================================

print("🔍 Setting up hyperparameter search...\n")
print("⚠️  With 20M records, we'll use a smart strategy:")
print("   1. Tune on 1M sample (fast)")
print("   2. Apply best params to full dataset\n")

# Create 1M sample for tuning
sample_size = 1_000_000
print(f"📊 Creating {sample_size:,} record sample for tuning...")

sample_indices = np.random.choice(X_train.index, size=min(sample_size, len(X_train)), replace=False)
X_train_sample = X_train.loc[sample_indices]
y_train_sample = y_train.loc[sample_indices]

print(f"✅ Sample created: {len(X_train_sample):,} records\n")

# Improved parameter grid (based on research for housing prices)
param_distributions = {
    'n_estimators': [300, 500, 800, 1000],           # More trees
    'max_depth': [5, 7, 9, 12, 15],                  # Deeper trees
    'learning_rate': [0.01, 0.03, 0.05, 0.08, 0.1], # Lower learning rates
    'subsample': [0.7, 0.8, 0.9],                    # Sample ratio
    'colsample_bytree': [0.7, 0.8, 0.9],            # Feature ratio
    'min_child_weight': [1, 3, 5],                   # Regularization
    'gamma': [0, 0.1, 0.2],                          # Pruning
    'reg_alpha': [0, 0.1, 0.5],                      # L1 regularization
    'reg_lambda': [1, 1.5, 2]                        # L2 regularization
}

print("📋 Optimized parameters to tune:")
for param, values in param_distributions.items():
    print(f"  • {param}: {values}")

# Create base model
xgb_model = XGBRegressor(random_state=42, n_jobs=-1, tree_method='hist')  # 'hist' = faster!

# RandomizedSearchCV on SAMPLE
print("\n⏳ Starting random search on 1M sample...")
print("Testing 20 combinations with 2-fold CV (faster but still reliable)...\n")

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=20,              # 20 combinations (good balance)
    scoring='r2',
    cv=2,                   # 2-fold (faster)
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit on SAMPLE
random_search.fit(X_train_sample, y_train_sample)

print("\n✅ Tuning complete on sample!")
print("\n🚀 Now training best model on FULL dataset...")

# Get best parameters
best_params = random_search.best_params_
print(f"\n🏆 Best parameters: {best_params}")

# Train final model on FULL dataset with best parameters
final_model = XGBRegressor(**best_params, random_state=42, n_jobs=-1, tree_method='hist')
final_model.fit(X_train, y_train)

print("\n✅ Final model trained on full dataset!")

🔍 Setting up hyperparameter search...

⚠️  With 20M records, we'll use a smart strategy:
   1. Tune on 1M sample (fast)
   2. Apply best params to full dataset

📊 Creating 1,000,000 record sample for tuning...
✅ Sample created: 1,000,000 records

📋 Optimized parameters to tune:
  • n_estimators: [300, 500, 800, 1000]
  • max_depth: [5, 7, 9, 12, 15]
  • learning_rate: [0.01, 0.03, 0.05, 0.08, 0.1]
  • subsample: [0.7, 0.8, 0.9]
  • colsample_bytree: [0.7, 0.8, 0.9]
  • min_child_weight: [1, 3, 5]
  • gamma: [0, 0.1, 0.2]
  • reg_alpha: [0, 0.1, 0.5]
  • reg_lambda: [1, 1.5, 2]

⏳ Starting random search on 1M sample...
Testing 20 combinations with 2-fold CV (faster but still reliable)...

Fitting 2 folds for each of 20 candidates, totalling 40 fits

✅ Tuning complete on sample!

🚀 Now training best model on FULL dataset...

🏆 Best parameters: {'subsample': 0.8, 'reg_lambda': 1.5, 'reg_alpha': 0.1, 'n_estimators': 800, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'gamma'

---

## 🏆 Step 6: Best Parameters & Performance

Let's see what parameters performed best and how much we improved!

In [11]:
# ==========================================
# Display Best Parameters and Results
# ==========================================

print("🏆 BEST PARAMETERS FOUND:")
print("="*60)
best_params = random_search.best_params_
for param, value in best_params.items():
    print(f"  {param:<20} = {value}")

print("\n" + "="*60)
print("📊 CROSS-VALIDATION RESULTS:")
print("="*60)
print(f"Best CV R² Score: {random_search.best_score_:.4f} ({random_search.best_score_*100:.2f}%)")

# Get best model
best_model = random_search.best_estimator_

# Test on holdout set
y_pred_tuned = best_model.predict(X_test)

tuned_r2 = r2_score(y_test, y_pred_tuned)
tuned_mae = mean_absolute_error(y_test, y_pred_tuned)
tuned_rmse = np.sqrt(mean_squared_error(y_test, y_pred_tuned))

print("\n" + "="*60)
print("📊 TEST SET PERFORMANCE (Tuned Model):")
print("="*60)
print(f"R² Score:                    {tuned_r2:.4f} ({tuned_r2*100:.2f}%)")
print(f"Mean Absolute Error (MAE):   £{tuned_mae:,.0f}")
print(f"Root Mean Squared Error:     £{tuned_rmse:,.0f}")
print("="*60)

# Compare with baseline
print("\n" + "="*60)
print("📈 IMPROVEMENT OVER BASELINE:")
print("="*60)
print(f"R² Improvement:     {baseline_r2:.2%} → {tuned_r2:.2%} (+{(tuned_r2-baseline_r2)*100:.2f}%)")
print(f"MAE Improvement:    £{baseline_mae:,.0f} → £{tuned_mae:,.0f} (-£{baseline_mae-tuned_mae:,.0f})")
print(f"RMSE Improvement:   £{baseline_rmse:,.0f} → £{tuned_rmse:,.0f} (-£{baseline_rmse-tuned_rmse:,.0f})")
print("="*60)

# Did we reach our goal?
print("\n🎯 TARGET ACHIEVEMENT:")
target_r2 = 0.70
target_mae = 45000
print(f"R² Target (70%):      {'✅ ACHIEVED!' if tuned_r2 >= target_r2 else f'❌ Not yet ({tuned_r2*100:.1f}%)'}")
print(f"MAE Target (<£45k):   {'✅ ACHIEVED!' if tuned_mae <= target_mae else f'❌ Not yet (£{tuned_mae:,.0f})'}")

🏆 BEST PARAMETERS FOUND:
  subsample            = 0.8
  reg_lambda           = 1.5
  reg_alpha            = 0.1
  n_estimators         = 800
  min_child_weight     = 1
  max_depth            = 7
  learning_rate        = 0.05
  gamma                = 0.2
  colsample_bytree     = 0.8

📊 CROSS-VALIDATION RESULTS:
Best CV R² Score: 0.5860 (58.60%)

📊 TEST SET PERFORMANCE (Tuned Model):
R² Score:                    0.5891 (58.91%)
Mean Absolute Error (MAE):   £51,971
Root Mean Squared Error:     £90,888

📈 IMPROVEMENT OVER BASELINE:
R² Improvement:     58.60% → 58.91% (+0.30%)
MAE Improvement:    £52,424 → £51,971 (-£453)
RMSE Improvement:   £91,221 → £90,888 (-£334)

🎯 TARGET ACHIEVEMENT:
R² Target (70%):      ❌ Not yet (58.9%)
MAE Target (<£45k):   ❌ Not yet (£51,971)
